In [ ]:
import numpy as np
import torch
import igl
import time

import meshplot as mp
import sys as _sys
_sys.path.append("../src")
from fem_system import *
from elastic_energy import *
from adjoint_sensitivity import *
from objectives import *
from harmonic_interpolator import *
from shape_optimizer import *
import matplotlib.pyplot as plt
from create_vol_cube import create_vol_cube

from utils import *

shadingOptions = {
    "flat":True,
    "wireframe":False,   
}

rot = torch.tensor(
    [[1.0,  0.0, 0.0],
     [0.0,  0.0, 1.0],
     [0.0, -1.0, 0.0]]
)

torch.set_default_dtype(torch.float64)

def to_numpy(tensor):
    return tensor.detach().clone().numpy()

# Create the deformed object

## Load the mesh

In [ ]:
# v, _, _, t, _, _ = igl.read_obj("../data/beam.obj")
v, t = create_vol_cube(8, 2, 2)
v[:, 0] *= 4
v = torch.tensor(v)
t = torch.tensor(t)

aabb = torch.max(v, dim=0).values - torch.min(v, dim=0).values
length_scale = torch.mean(aabb)

be = igl.edges(igl.boundary_facets(to_numpy(t)))
e = igl.edges(to_numpy(t))
bv = np.unique(igl.boundary_facets(to_numpy(t))).astype(np.int64)
iv = np.array([idx for idx in range(v.shape[0]) if not idx in bv]).astype(np.int64)
convertBV = {bv[i].item():i for i in range(bv.shape[0])}
beTarget = np.array([[convertBV[bEdge[0]], convertBV[bEdge[1]]] for bEdge in be])

# p = mp.plot(to_numpy(v @ rot.T), to_numpy(t), shading=shadingOptions)

## Add some physical characteristics

In [ ]:
rho     = 5e2 # [kg.m-3]
damping = 0.0
young   = 2e5 # [Pa] 
poisson = 0.2

# Find some of the lowest vertices and pin them
minX    = torch.min(v[:, 0])
pin_idx = list(torch.arange(v.shape[0])[v[:, 0] < minX + 0.2*aabb[0]])
vIdx  = np.arange(v.shape[0])
pin_idx  = vIdx[np.in1d(vIdx, bv) & np.in1d(vIdx, pin_idx)]
print("Pinned vertices: {}".format(pin_idx))

# Initial guess

The idea is that we start deforming the mesh by inverting gravity.

In [ ]:
# Inverted gravity
force_mass = torch.zeros(size=(3,))
force_mass[2] = + 9.81

# Gravity going in the wrong direction

ee = NeoHookeanElasticEnergy(young, poisson)
v = HarmonicInterpolator(v, t, iv).interpolate(v[bv])
solid_init = FEMSystem(v, t, ee, rho=rho, pin_idx=pin_idx, f_mass=force_mass)

v_init_eq = solid_init.find_equilibrium(v)
plot_torch_solid(solid_init, v_init_eq, be, rot, length_scale)

# Use these as initial guesses
v_init_rest = v_init_eq.clone().detach()
v_init_def  = solid_init.v_rest.clone().detach()

# v_init_rest = solid_init.v_rest.clone().detach()
# v_init_def  = solid_init.v_def.clone().detach()

# Inverse design


In [ ]:
force_mass = torch.zeros(size=(3,))
force_mass[2] = - 9.81
use_linear  = False

# The target is the initial raw mesh
vt_surf = v[bv, :].clone()

# Create solid
if use_linear:
    ee = LinearElasticEnergy(young, poisson)
else:
    ee = NeoHookeanElasticEnergy(young, poisson)
solid_ = FEMSystem(v_init_rest, t, ee, rho=rho, pin_idx=pin_idx, f_mass=force_mass)

optimizer = ShapeOptimizer(solid_, v_init_def, vt_surf, bv, be, beTarget, weight_reg=1.0, force_thresh=1e-3)

v_eq_init = optimizer.v_eq.clone().detach() #bookkeeping

In [ ]:
initial_rest = optimizer.solid.v_rest
initial_eq = optimizer.v_eq

In [ ]:
optimizer.optimize(step_size_init=1.0e-2, max_l_iter=10, n_optim_steps=100)

In [ ]:
final_rest = optimizer.solid.v_rest
final_eq = optimizer.v_eq

In [ ]:
p = mp.plot(to_numpy(optimizer.v_eq @ rot.T), to_numpy(optimizer.solid.tet), shading=shadingOptions)
p.add_edges(to_numpy(initial_rest @ rot.T), be, shading={"line_color": 'blue'})
p.add_edges(to_numpy(final_rest @ rot.T), be, shading={"line_color": 'red'})

### Before optimization

In [ ]:
force_mass = torch.zeros(size=(3,))
force_mass[2] = - 9.81

# Gravity going in the wrong direction

ee = NeoHookeanElasticEnergy(young, poisson)
v = HarmonicInterpolator(v, t, iv).interpolate(v[bv])
solid_init = FEMSystem(v, t, ee, rho=rho, pin_idx=pin_idx, f_mass=force_mass)

solid_init.update_rest_shape(initial_rest)
v_init_eq = solid_init.find_equilibrium(v)
plot_torch_solid(solid_init, v_init_eq, be, rot, length_scale)

### After optimization

In [ ]:
force_mass = torch.zeros(size=(3,))
force_mass[2] = - 9.81

# Gravity going in the wrong direction

ee = NeoHookeanElasticEnergy(young, poisson)
v = HarmonicInterpolator(v, t, iv).interpolate(v[bv])
solid_init = FEMSystem(v, t, ee, rho=rho, pin_idx=pin_idx, f_mass=force_mass)

solid_init.update_rest_shape(final_rest)
v_init_eq = solid_init.find_equilibrium(v)
plot_torch_solid(solid_init, v_init_eq, be, rot, length_scale)

In [ ]:
plt.figure(figsize=(10, 6))
plt.plot(to_numpy(optimizer.objectives[optimizer.objectives > 0]),label="Full objective")
plt.plot(to_numpy(optimizer.obj_target[optimizer.objectives > 0]),label="Target objective")
plt.plot(to_numpy(optimizer.obj_reg[optimizer.objectives > 0]),label="Regularization objective")
plt.title("Objective as optimization goes", fontsize=14)
plt.xlabel("Optimization steps", fontsize=12)
plt.ylabel("Objective", fontsize=12)
plt.legend()
plt.grid()
plt.show()

Green (Initial guess for rest state) deploys to Black

Blue (Optimized rest state) deploys to Yellow

Red is the Target Shape


In [ ]:
p = mp.plot(to_numpy(optimizer.v_eq @ rot.T), to_numpy(t), shading=shadingOptions)
# p.add_points(np.array(optimizer.solid.v_def)[pin_idx, :] @ rot.T, shading={"point_color":"black", "point_size": 0.2})
p.add_edges(np.array(v_init_rest @ rot.T), be, shading={"line_color": "green"})
p.add_edges(v @ rot.T, be, shading={"line_color": "red"})
p.add_edges(to_numpy(v_eq_init @ rot.T), be, shading={"line_color": "black"})
p.add_edges(to_numpy(optimizer.solid.v_rest @ rot.T), be, shading={"line_color": "blue"})


In [ ]:
v_rest_optim_g = optimizer.solid.v_rest.clone().detach() #bookkeeping

## Add point load to the right most vertices


In [ ]:
maxX        = torch.min(v[:, 0])
f_point_idx = torch.arange(v.shape[0])[v[:, 0] > maxX - 0.01*aabb[0]]

f_point = torch.zeros(size=(f_point_idx.shape[0], 3))
f_point[:, 2] = -100

optimizer.solid.add_point_load(f_point_idx, f_point)
optimizer.set_params(optimizer.params)
v_def_optim_g_under_point = optimizer.v_eq.clone().detach() #bookkeeping

optimizer.reset_BFGS()
optimizer.reset_plot()

In [ ]:
optimizer.optimize(step_size_init=2.0e-2, max_l_iter=10, n_optim_steps=100)

Green (Optimum rest state under gravity) deploys to Black with the additional point load

Blue (Optimized rest state) deploys to Yellow

Red is the Target Shape


In [ ]:
p = mp.plot(to_numpy(optimizer.v_eq @ rot.T), to_numpy(t), shading=shadingOptions)
# p.add_points(np.array(optimizer.solid.v_def)[pin_idx, :] @ rot.T, shading={"point_color":"black", "point_size": 0.2})
p.add_edges(to_numpy(v_rest_optim_g @ rot.T), be, shading={"line_color": "green"})
p.add_edges(v @ rot.T, be, shading={"line_color": "red"})
p.add_edges(to_numpy(v_def_optim_g_under_point @ rot.T), be, shading={"line_color": "black"})
p.add_edges(to_numpy(optimizer.solid.v_rest @ rot.T), be, shading={"line_color": "blue"})
